## preguntar por PostGres entorno -  cargamos el pandas 

In [1]:
import pandas as pd
import numpy as np

para poder pintar con plot pylab #esto permite a pandas pintar en el notebook 

In [2]:
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [3]:
train = pd.read_csv('C:/Users/Juanpe/MachineLearningTest/data/train.csv')
test = pd.read_csv('C:/Users/Juanpe/MachineLearningTest/data/test.csv')

In [ ]:
train.head(5)

In [ ]:
list(train)

In [ ]:
train.age.plot()

In [ ]:
train['age'].hist()

In [ ]:
train.hist(figsize=(20,20))

In [ ]:
train.boxplot('age')

In [ ]:
train.describe()

In [ ]:
train.iloc[21,2]

In [ ]:
train.loc[21,'age']

In [ ]:
train.shape

In [ ]:
train.info()

In [4]:
nrow = train.shape[0]

In [ ]:
len(train)

In [ ]:
train.num.unique()

vamos a hacer un for

In [5]:
for i in range(nrow):
    if train['num'][i] != 0:
        train.loc[i,'num'] = 1
        

In [ ]:
train

In [ ]:
range(nrow)

In [ ]:
train['num'][train['num']!=0]

In [ ]:
train.loc[2,'num']

In [ ]:
train

In [ ]:
train[train['num']!=0]

## Preprocessing 

http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values

#nos saltamos la estandarización
#vemos valores Nan

In [ ]:
pd.isnull(train).any(1).nonzero()[0]

In [ ]:
train.iloc[pd.isnull(train).any(1).nonzero()[0]]

In [ ]:
train.loc[[87, 166, 192],:]

#pillamos la librería q imputa valores

In [6]:
train = train.fillna(train.mean())#forma simplificada de Juan

In [ ]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(train)

nos crea una matriz sin nombres de campo, por eso lo de columns

In [ ]:
train = pd.DataFrame(imp.transform(train),columns=train.columns)

In [ ]:
train

## vamos a ver cada variable % sanos

In [ ]:
train.groupby('num').size()

In [ ]:
train.num.mean() #hay un 43% de enfermos

In [ ]:
train.sex.mean() #hay un 67% de hombres

In [ ]:
train.groupby('sex').agg({'num': np.mean}) #sexo importa

In [ ]:
train.groupby('cp').agg({'num': np.mean}) # cp importa mucho

In [ ]:
train.groupby('fbs').agg({'num': np.mean}) #nada castaña pilonga

In [ ]:
train.groupby('restecg').agg({'num': np.mean}) # importa poco

In [ ]:
train.groupby('exang').agg({'num': np.mean}) # importa bastante

In [ ]:
train.groupby('slope').agg({'num': np.mean}) # importa bastante

In [ ]:
train.groupby('ca').agg({'num': np.mean}) #importa demasiado!!

In [ ]:
train.groupby('thal').agg({'num': np.mean}) # importa bastante

## voy a pintar un mapa de correlaciones de variables

In [ ]:
#correlation map
import seaborn as sns
import matplotlib.pyplot as plt
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

sale lo q habia visto yo con los group by!!!!!!

http://python-apuntes.blogspot.com.es/2016/07/importancia-variables.html

## lanzamos modelo naive bayes

separamos los datos X, y (lo pasamos a forma matriz por Victor manía)

In [7]:
X, y = train.iloc[:,:-1].as_matrix(), train.iloc[:,-1].as_matrix()

y tb en train y test de ambas variables

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train


## lanzamos modelo. copio y pego del scikit learn la forma

In [28]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test) 

#print("Number of mislabeled points out of a total %d points : %d"% (X_train.shape[0],(y_train != y_pred).sum())) Number of mislabeled points out of a total 150 points : 6

## vemos la matriz de confusión

In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)

In [41]:
cnf_matrix

array([[39,  5],
       [ 6, 38]], dtype=int64)

In [49]:
np.concatenate(cnf_matrix) 

array([39,  5,  6, 38], dtype=int64)

In [51]:
pd.DataFrame(np.concatenate(cnf_matrix)).T  #TN FP FN TP

,0,1,2,3
0,39,5,6,38


In [76]:
pd.DataFrame(np.concatenate(cnf_matrix))

,0
0,39
1,5
2,6
3,38


In [ ]:
pd.DataFrame(cnf_matrix,columns=['no','si'], index=['no','si'])

In [ ]:
gnb.get_params()

## qué peso da el modelo a cada variable?  
http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

In [ ]:
gnb.score(X_test, y_test)

In [ ]:
gnb.score(X_train, y_train)

## vamos a mejorar el modelo. hay muchas formas. vamos a hacer 2 de ellas. 1) PCA - analisis de componentes principales 2) Iterative train-test scikit learn

## 1) PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(9) # lo vamos a poner en 9 porq a Pablo le ha salido bien así, pero lo suyo es hacerlo iterativo

Xtr =pca.fit(X).transform(X)

Xtr_train, Xtr_test, ytr_train, ytr_test = train_test_split(Xtr, y, test_size=0.33)

In [ ]:
#lanzamos modelo
gnbtr = GaussianNB()

ytr_pred = gnb.fit(Xtr_train, ytr_train).predict(Xtr_test) 

In [ ]:
# Compute confusion matrix
cnftr_matrix = confusion_matrix(ytr_test, ytr_pred)
confussiontr = pd.DataFrame(cnftr_matrix,columns=['no','si'], index=['no','si'])
confussiontr


## 2) train-test iterativo

lo q queremos es comparar la predicción del modelo en diferentes separaciones train-test. lo guardamos en results los valores y en a_elegir los falsos positivos, etc

In [52]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

def traintest (porcentaje, iteraciones):
    results = []
    a_elegir = []
    for i in range(iteraciones):
        # dividimos en train-test  
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= porcentaje)#quitamos el random_state

        # lanzamos modelo
        gnb = GaussianNB()
        y_pred = gnb.fit(X_train, y_train).predict(X_test)

        # Compute confusion matrix
        cnf_matrix = confusion_matrix(y_test, y_pred)

        # guardamos matriz
        matriz = pd.DataFrame(cnf_matrix,columns=['no','si'], index=['no','si'])

        results.append([matriz, y_pred, y_test, X_test, y_train, X_train])
        a_elegir.append([porcentaje, pd.DataFrame(np.concatenate(cnf_matrix)).T])#TN FP FN TP
    return results, a_elegir

In [53]:
RES = traintest(0.33,10)

vamos a registrar el resultado del modelo para elegir el q nos parezca más apropiado. ¿apropiado? por falsos negativos, falsos positivos... 

In [100]:
RES[-1:] #a_elegir

([[0.33,     0  1  2   3
   0  42  9  8  29], [0.33,     0   1   2   3
   0  42  10  10  26], [0.33,     0  1   2   3
   0  46  1  11  30], [0.33,     0  1  2   3
   0  41  6  5  36], [0.33,     0  1   2   3
   0  45  7  12  24], [0.33,     0  1  2   3
   0  43  5  6  34], [0.33,     0  1  2   3
   0  44  9  5  30], [0.33,     0  1   2   3
   0  39  7  11  31], [0.33,     0  1  2   3
   0  41  9  9  29], [0.33,     0  1  2   3
   0  47  5  8  28]],)

In [99]:
list(RES[-1:])

[[[0.33,     0  1  2   3
   0  42  9  8  29], [0.33,     0   1   2   3
   0  42  10  10  26], [0.33,     0  1   2   3
   0  46  1  11  30], [0.33,     0  1  2   3
   0  41  6  5  36], [0.33,     0  1   2   3
   0  45  7  12  24], [0.33,     0  1  2   3
   0  43  5  6  34], [0.33,     0  1  2   3
   0  44  9  5  30], [0.33,     0  1   2   3
   0  39  7  11  31], [0.33,     0  1  2   3
   0  41  9  9  29], [0.33,     0  1  2   3
   0  47  5  8  28]]]

In [92]:
hola2 = pd.DataFrame(list(RES[-1:]))

In [93]:
hola2

,0,1,2,3,4,5,6,7,8,9
0,"[0.33, [0]]","[0.33, [0]]","[0.33, [0]]","[0.33, [0]]","[0.33, [0]]","[0.33, [0]]","[0.33, [0]]","[0.33, [0]]","[0.33, [0]]","[0.33, [0]]"


In [13]:
a_elegir = [porcentaje, RES]

[[    no  si
  no  37   7
  si   8  36,
  array([0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
         0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
         1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
         0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1], dtype=int64),
  array([1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
         0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
         0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1], dtype=int64),
  array([[ 48.,   1.,   4., ...,   2.,   0.,   7.],
         [ 63.,   1.,   4., ...,   2.,   1.,   7.],
         [ 64.,   1.,   3., ...,   2.,   0.,   7.],
         ..., 
         [ 57.,   1.,   3., ...,   1.,   1.,   7.],
         [ 55.,   1.,   4., ...,   2.,   1.,   7.],
         [ 60.,   1.,   4., ...,   2.,   2.,   7.]]),
 

In [11]:
for i in range(len(RES)):
    print(RES[i][0])

    no  si
no  37   7
si   8  36
    no  si
no  41   7
si   7  33
    no  si
no  50   4
si   5  29
    no  si
no  49   4
si   9  26
    no  si
no  41   7
si   8  32
    no  si
no  48   4
si   8  28
    no  si
no  44   5
si   8  31
    no  si
no  47   5
si   7  29
    no  si
no  44   4
si   6  34
    no  si
no  38   9
si   8  33


## voy a ver el resultado del modelo con el scalado q hace JuanCB

este es el código de Juan:

dataTrain = dataTrain.fillna(dataTrain.mean())

dataTest = dataTest.replace('?', np.nan)
dataTest = dataTest.fillna(dataTest.mean())
dataTest = dataTest.dropna()

list_features = list(dataTrain)
variable2predict = 'num'

list_features.remove(variable2predict)
#display(columnas)

features_gen = pd.Index(list_features)
display(features_gen)

dataTotal = dataTrain[features_gen].append(dataTest[features_gen])

dataTotal = pd.DataFrame(scale(dataTotal))

train_X = dataTotal[0:dataTrain.shape[0]].as_matrix()
test_X = dataTotal[dataTrain.shape[0]::].as_matrix()
train_y = dataTrain['num']

model = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, train_y)
predictions = model.predict(test_X)

In [ ]:
dataTrain = train.fillna(train.mean())

dataTest = test.replace('?', np.nan) 
dataTest = dataTest.fillna(test.mean()) 
dataTest = dataTest.dropna()

nrow = dataTrain.shape[0]
for i in range(nrow):
    if dataTrain['num'][i] != 0:
        dataTrain.loc[i,'num'] = 1
        
list_features = list(dataTrain) 
variable2predict = 'num'

list_features.remove(variable2predict)

#display(columnas)

features_gen = pd.Index(list_features) 

from sklearn.preprocessing import scale
dataTotal = dataTrain[features_gen].append(dataTest[features_gen])

dataTotal = pd.DataFrame(scale(dataTotal))

train_X = dataTotal[0:dataTrain.shape[0]].as_matrix()
test_X = dataTotal[dataTrain.shape[0]::].as_matrix()
train_y = dataTrain['num']

def traintest_juan (porcentaje, iteraciones):
    results = []
    for i in range(iteraciones):
        # dividimos en train-test  
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= porcentaje)#quitamos el random_state

        # lanzamos modelo
        gnb = GaussianNB()
        y_pred = gnb.fit(X_train, y_train).predict(X_test)

        # Compute confusion matrix
        cnf_matrix = confusion_matrix(y_test, y_pred)

        # guardamos matriz
        matriz = pd.DataFrame(cnf_matrix,columns=['no','si'], index=['no','si'])

        results.append([matriz, y_pred, y_test, X_test, y_train, X_train])
    return results
    
RES2 = traintest_juan(0.33,10)

    

In [ ]:
for i in range(len(RES)):
    print(RES2[i][0] +  RES[i][0]) # me ha dado un salto para comparar

In [ ]:
matriz